# **7. cvičení - Lokální zarovnání (25.3.2021)**

## Zarovnávání sekvencí

* vstup: 2 sekvence, skórovací systém
* výstup: uspořádání co nejdelšího úseku 2 sekvencí, jehož skóre je maximální pro daný skórovací systém
* Smith-Waterman algoritmus


**Úkol č. 1:** <br>
a) Co potřebujeme, pokud chceme zarovnávat sekvence pomocí lokálního zarovnání? Co je výsledkem zarovnání?<br>
b) Jaké sekvence je vhodné zarovnávat lokálně?<br>
c) Jak se vyplňuje první sloupec/řádek u matice S?<br>
d) Kde v matici S začíná zpětná cesta a kde končí? <br>
e) Jak se liší lokální zarovnání od globálního zarovnání?

**Odpověď:**
<br/>a) Potřebujeme dvě sekvence a skórovací systém. Výsledkem bude zarovnání nejvice si podobných úseků.<br>
b) Sekvence, které jsou si podobné jenom v určítých úsecích.<br>
c) Vyplňuje se nulami.<br>
d) Zpětná cesta se začíná od největší hodnoty v matici až po první nulu.<br>
e) Výsledkem globálního zarovnání jsou dvě celé zarovnáné sekvence, víceméně u lokálního zarovnání můžeme zarovnávat jenom nejvíce podobné úseky.

## Smith-Waterman algoritmus
* vhodné pro sekence podobné si pouze v určitém úseku
* rozdíly oproti Needleman-Wunsch algoritmu:
  * inicializace 1. řádku a sloupce pouze hodnotami 0
  * nahrazení záporného maxima hodnotou 0
  * zpětná cesta začíná od maximální hodnoty v matici ohodnocení
  * zpětná cesta končí v prvku s 0







* sekvence *A* má délku *i* a sekvence *B* má délku *n*
* matice ohodnocení *S* má velikost *m* + 1, *n* + 1
* první řádek a sloupec matice *S* obsahuje 0
* aktuální prvek *S* na pozici [i, j] je počítán z předcházejících hodnot:
  * prvek z řádku [i-1, j]
  * prvek ze sloupce [i, j-1]
  * prvek diagonální [i-1, j-1]
* vybíráme maximum z okolních prvků a z 0  


<img src="https://drive.google.com/uc?export=view&id=1DZ5LMwc-UfkDJbej0LuNv4kQ0prHTp-n" width="500">

### Příklad:
* sekvence: GATAC, GTA
* skórovací systém: match = 2, mismatch = 1, gap = -1
* vyplňujeme matici *S*
* zarovnání je určeno zpětnou cestou
* zpětná cesta začíná od maximálního prvku matice *S* a končí v prvku s 0
* následujeme šipky
* zarovnání:

<img src="https://drive.google.com/uc?export=view&id=17ZBTB93rf_Jf_ZplU3UISmdyTQZMXM7w" width="300">

<img src="https://drive.google.com/uc?export=view&id=
1aGOIp8B_b8D50JGUYCjUwtgTyA2h16ud" width="400">

**Úkol č. 2:** <br>
Seznamte se s afinními mezerami – co jsou to afinní mezery, jaké matice se u afinních mezer počítají, jak funguje zpětná cesta. Využijte slidy z přednášky, dostupnou literaturu či výuková videa na youtube.

**Odpověď:**
<br/>Afinní mezery jsou navazující se na sebe mezery.
Musíme navíc spočítat matici delece (D) a matici inzerce (I).
Zpětná cesta se začíná v pravém dolním rohu a končí se v levém horním. Cesta se pokračuje podle toho z jaké matice bylo výbráno maximum.

**Úkol č. 3:** <br>
Vytvořte vlastní funkci pro výpočet lokálního zarovnání 2 nukleotidových sekvencí. Vstupem funkce budou 2 nukleotidové sekvence a skórovací systém, výstupem funkce bude matice ohodnocení a zarovnané sekvence.


In [ ]:
pip install biopython

     |████████████████████████████████| 2.3MB 15.1MB/s 


In [ ]:
import numpy as np
from Bio import Seq
from Bio import SeqIO
from Bio import SeqRecord
from Bio import pairwise2
from Bio.pairwise2 import format_alignment


def lokalni_zarovnani (seq1, seq2, skore):

  m = len(seq1)
  n = len(seq2)

  # vytvoříme nulovou matici ohodnocení S o rozměrech m+1, n+1
  S = np.zeros((m+1, n+1), dtype=int)
  # pomocí cyklů procházíme matici a vyplňujeme ji
  for i in range(1, m+1):
    for j in range(1,n+1):
      if seq1[i-1] == seq2[j-1]:
        S[i][j] = max(0, (S[i-1][j] + skore[2]), (S[i][j-1] + skore[2]), (S[i-1][j-1] + skore[0]))
      else:
        S[i][j] = max(0, (S[i-1][j] + skore[2]), (S[i][j-1] + skore[2]), (S[i-1][j-1] + skore[1]))


  # zobrazíme si matici ohodnocení S
  print('Matice ohodnocení S je:')
  print(S)

  ## tvorba zpětné cesty

  # nalezení maximální hodnoty v matici S
  maxi = 0
  maxj = 0
  value = 0

  for i in range(1, m+1): 
    for j in range(1, n+1):
      if S[i][j] > value:
        maxi = i
        maxj = j
        value = S[i][j]
  # vytvoříme si prázdné proměnné, do kterých budeme ukládat zarovnání
  alignseq1 = ''
  alignseq2 = ''
  i = maxi
  j = maxj
  # vytvoříme si cyklus pro zpětný pohyb v matici a který bude ukončen, pokud na pozicích řádek, sloupec je 0
  while True:
    
    if S[i][j] == 0:
      break

      # pokud jsou shodné znaky, budeme počítat s hodnotou match
      if seq1[i-1] == seq2[j-1]:
        m = skore[0]
      # pokud jsou rozdílné znaky, budeme počítat s hodnotou mismatch
      else:
        m = skore[1]
      # najdeme, kde je v okolí daného prvku, na kterém se nacházíme, maximální hodnota
      maximum = max((S[i-1][j] + skore[2]), (S[i][j-1] + skore[2]), (S[i-1][j-1] + m))

      # pokud bylo maximum na diagonále, posuneme se doleva a nahoru a uložíme příslušné znaky do zarovnání
      if maximum == (S[i-1][j-1] + m):
        alignseq1 = seq2[j-1] + alignseq1
        alignseq2 = seq1[i-1] + alignseq2
        i -=1
        j -=1
        continue

      # pokud bylo maximum nahoře, posuneme se nahoru a uložíme příslušný znak a pomlčku do zarovnání
      if maximum == (S[i-1][j] + skore[2]):
        alignseq1 = '-' + alignseq1
        alignseq2 = seq1[i-1] + alignseq2
        i -=1
        continue

      # pokud bylo maximum vlevo, posuneme se doleva a uložíme příslušný znak a pomlčku do zarovnání
      if maximum == (S[i][j-1] + skore[2]):
        alignseq1 = seq2[j-1] + alignseq1
        alignseq2 = '-' + alignseq2
        j -=1
        continue
  # zobrazení zarovnání
  print('\nZarovnáni je:')
  print(alignseq1)
  print(alignseq2)
# volání funkce
lokalni_zarovnani("AGC", "AATGC", [2, -1, -2])

Matice ohodnocení S je:
[[0 0 0 0 0 0]
 [0 2 2 0 0 0]
 [0 0 1 1 2 0]
 [0 0 0 0 0 4]]


**Vstup:** <br>
("AGC", "AATGC", 2, -1, -2) <br>

**Výstup:**<br>
Matice ohodnocení S je: </br>  [[0. 0. 0. 0. 0. 0.]<br>
 [0. 2. 2. 0. 0. 0.]<br>
 [0. 0. 1. 1. 2. 0.]<br>
 [0. 0. 0. 0. 0. 4.]]<br>

Zarovnání je:  
GC </br>
GC